<a href="https://colab.research.google.com/github/Jass1313c/comp215/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import datetime, json, requests
from pprint import pprint

param = { "format": "geojson", "starttime": "2024-03-14","limit": 20000, "endtime": "2024-06-15"}
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
response = requests.get(url, params=param)
data = json.loads(response.text)

pprint(f"Count: {data['metadata']['count']}")

pprint(data)

pprint(f"Count: {data['metadata']['count']}")

for feature in data['features']:
  if "Texas" in feature['properties']['place']:
    Latitude = feature['geometry']['coordinates'][1]
    Longitude = feature['geometry']['coordinates'][0]
    print(f"Mag: {feature['properties']['mag']},   Place: {feature['properties']['place']},   Time: {datetime.datetime.fromtimestamp(feature['properties']['time'] / 1000)},   Lat: {Latitude},   Long: {Longitude}")

Streaming output truncated to the last 5000 lines.
                              'rms': 0.05,
                              'sig': 18,
                              'sources': ',nc,',
                              'status': 'reviewed',
                              'time': 1714588449030,
                              'title': 'M 1.1 - 7 km NW of The Geysers, CA',
                              'tsunami': 0,
                              'type': 'earthquake',
                              'types': ',focal-mechanism,nearby-cities,origin,phase-data,',
                              'tz': None,
                              'updated': 1715366710974,
                              'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc74043876'},
               'type': 'Feature'},
              {'geometry': {'coordinates': [-149.5828, 63.1364, 86.4],
                            'type': 'Point'},
               'id': 'ak0245m2h0m3',
               'properties': {'alert': None,
            